In [ ]:
%load_ext autoreload
%autoreload 2
import sys

# instead of creating a package using setup.py or building from a docker/singularity file,
# import the sister directory of src code to be called on in notebook.
# This keeps the notebook free from code to only hold visualizations and is easier to test
# It also helps keep the state of variables clean such that cells aren't run out of order with a mysterious state
sys.path.append("..")

In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import skew
import statistics
import cartopy.crs as crs
import cartopy.feature as cfeature
import xarray as xr
import glob

In [ ]:
def format_df(df):
    """Format a DataFrame with counts of values into a DataFrame with individual values
    
    Args:
    - df: A pandas DataFrame with columns "COUNT" and "VALUE"
    
    Returns:
    - A new pandas DataFrame with a single column "VALUE" containing individual values
    
    Example:
    input DataFrame:
    
    | COUNT | VALUE |
    |-------|-------|
    |   2   |  10   |
    |   3   |  20   |
    |   1   |  30   |
    
    output DataFrame:
    
    | VALUE |
    |-------|
    |  10   |
    |  10   |
    |  20   |
    |  20   |
    |  20   |
    |  30   |
    """
    
    new_df = pd.DataFrame()
    value_list = []
    
    # iterate over the rows of the input DataFrame
    for x, _ in df.iterrows():
        
        # extract the count and value from each row
        count = int(df.iloc[x]["COUNT"])
        value = df.iloc[x]["VALUE"]
        
        # repeat the value the specified number of times
        for n in np.arange(count):
            val = value
            value_list.append(val)
    
    # create a new DataFrame with the individual values
    new_df["VALUE"] = value_list
    
    return new_df



def stat_anal(directory, state_df, station_list, lonlist, latlist):
    """
    Performs statistical analysis on a set of elevation data for a given state.

    Args:
    directory (str): File directory where the elevation data is stored.
    state_df (pandas DataFrame): DataFrame containing the elevation data for each station in the state.
    station_list (list): List of station IDs.
    lonlist (list): List of longitude coordinates for each station.
    latlist (list): List of latitude coordinates for each station.

    Returns:
    final_df (pandas DataFrame): DataFrame containing the results of the statistical analysis.
    Columns include: "station", "elev", "std", "variance", "skew", "med_dist", "lon", and "lat".
    """

    final_df = pd.DataFrame()
    std_list = []
    variance_list = []
    skew_list = []
    distance_list = []
    stations = []
    elevs = []
    x = 0
    for i in np.arange(1, 127):
        # read in csv
        df2 = pd.DataFrame()
        elev_df = pd.read_csv(
            f"{directory}/gfs/aspect_csv_{i}.csv"
        )
        dfv1 = format_df(elev_df)  # apply format_df to the elevation data
        std = statistics.stdev(dfv1["VALUE"])  # calculate the standard deviation
        variance = statistics.pvariance(dfv1["VALUE"])  # calculate the variance
        my_skew = skew(dfv1["VALUE"])  # calculate the skewness
        elevation = state_df["elev"].iloc[x]  # get the elevation for the current station
        station = station_list[x]  # get the station ID for the current station
        split_diff = dfv1["VALUE"] - state_df["elev"].iloc[x]  # calculate the difference between elevation and state_df
        diff_list = split_diff.to_list()  # convert the difference to a list
        df2["diff_elev"] = diff_list  # add the difference to the DataFrame
        describe = df2["diff_elev"].describe()  # calculate the descriptive statistics for the difference
        fifty = describe[5]  # get the median of the difference
        distance = state_df["elev"].iloc[x] - fifty  # calculate the median distance
        # add data to lists
        stations.append(station)
        elevs.append(elevation)
        distance_list.append(distance)
        skew_list.append(my_skew)
        variance_list.append(variance)
        std_list.append(std)
        x += 1

    final_df["station"] = stations
    final_df["elev"] = elevs
    final_df["std"] = std_list
    final_df["variance"] = variance_list
    final_df["skew"] = skew_list
    final_df["med_dist"] = distance_list
    final_df["lon"] = lonlist
    final_df["lat"] = latlist
    return final_df


In [ ]:
def current_time_mesonet_df(mesonet_data_path) -> pd.DataFrame:
    """
    This will return a dataframe that contains data from the mesonet sites

    Args:
        Mesonet Data Path (f string)

    Returns:
        df (pd.DataFrame): Mesonet Data Frame
    """

    # most recent year
    dir_Year = os.listdir(f"{mesonet_data_path}")
    sort_dir_Year = sorted(dir_Year)
    data_point_Year = sort_dir_Year[-1]

    # find most recent month
    dir_Month = os.listdir(f"{mesonet_data_path}/{data_point_Year}")
    sort_dir_Month = sorted(dir_Month)
    data_point_Month = sort_dir_Month[-1]

    # this is your directory for most recent year and month
    most_recent = os.listdir(
        f"{mesonet_data_path}/{data_point_Year}/{data_point_Month}"
    )

    # most recent datapoint
    sort_most_recent = sorted(most_recent)
    data_point = sort_most_recent[-1]

    # this will return the year of the most recent data point
    new_year = data_point[0:4]

    # this will return the month of the most recent datapoint
    new_month = data_point[4:6]

    # this will return the day of the most recent datapoint
    new_day = data_point[6:8]

    # create Mesonet DataFrame

    # year
    year = new_year

    # month
    month = new_month

    # day
    day = new_day

    # file path
    file = year + month + day + ".nc"

    mesonet_df = (
        xr.open_dataset(f"{mesonet_data_path}/{year}/{month}/{file}")
        .to_dataframe()
        .reset_index()
    )
    return mesonet_df

In [ ]:
def most_recent_time(df: pd.DataFrame, mesonet_data_path) -> pd.DataFrame:
    """
    This will return a dataframe that contains only the timestamps with filled data from the mesonet sites

    Args:
    Mesonet Data Path (f string)

    Returns:
    df (pd.DataFrame): Mesonet Data Frame
    """

    # most recent year
    dir_Year = os.listdir(f"{mesonet_data_path}")
    sort_dir_Year = sorted(dir_Year)
    data_point_Year = sort_dir_Year[-1]

    # find most recent month
    dir_Month = os.listdir(f"{mesonet_data_path}/{data_point_Year}")
    sort_dir_Month = sorted(dir_Month)
    data_point_Month = sort_dir_Month[-1]

    # this is your directory for most recent year and month
    most_recent = os.listdir(
        f"{mesonet_data_path}/{data_point_Year}/{data_point_Month}"
    )

    # most recent datapoint
    sort_most_recent = sorted(most_recent)
    data_point = sort_most_recent[-1]

    # this will return the year of the most recent data point
    new_year = data_point[0:4]

    # this will return the month of the most recent datapoint
    new_month = data_point[4:6]

    # this will return the day of the most recent datapoint
    new_day = data_point[6:8]

    # create Mesonet DataFrame

    # year
    year = new_year

    # month
    month = new_month

    # day
    day = new_day

    current_time_df = df.dropna(subset=["tair"])

    last_value = current_time_df["time_5M"].iat[-1]
    hour = last_value.hour
    minute = last_value.minute
    second = last_value.second

    string_hour = str(hour)
    string_minute = str(minute)
    string_sec = str(second)

    # time
    time = string_hour + ":" + string_minute + ":" + string_sec
    df.reset_index(inplace=True)

    # creating a new dataframe that is centered on the location in the dataframe
    mesonet_single_datetime_df = df.loc[df["time_5M"] == f"{year}-{month}-{day} {time}"]
    return mesonet_single_datetime_df

In [ ]:
# This will return the most recent data avail on mesonet
# this is my file path
ny_df = pd.read_csv("/home/aevans/nwp_bias/src/landtype/notebooks/nysm_coords.csv")

In [ ]:
# This will return the most recent data avail on mesonet
# this is my file path
ny_mesonet_data_path = "/home/aevans/nysm/archive/nysm/netcdf/proc"

In [ ]:
nysm_df1 = current_time_mesonet_df(ny_mesonet_data_path)
nysm_df = most_recent_time(nysm_df1, ny_mesonet_data_path)

In [ ]:
nysm_df

In [ ]:
ny_df["elev"] = nysm_df["elev"].to_list()
ny_df

In [ ]:
directory = os.listdir(f"/home/aevans/nwp_bias/src/landtype/elevation/data/NY/elev/nam")
sorted_direct = sorted(directory)

In [ ]:
sorted_direct

In [ ]:
# paths to data
path_ny = f"/home/aevans/nwp_bias/src/landtype/elevation/data/CSVs_elevation_ny_gfs"

In [ ]:
station_list_ny = ny_df["station"].to_list()
ny_df_lons = ny_df["longitude"].to_list()
ny_df_lats = ny_df["latitude"].to_list()

In [ ]:
x = 0
for i in range(1, 127):
    df = pd.read_csv(
        f"/home/aevans/nwp_bias/src/landtype/elevation/data/CSVs_slope_ny_gfs/aspect_csv_{i}.csv"
    )
    df.to_csv(
        f"/home/aevans/nwp_bias/src/landtype/elevation/data/NY/elev/gfs/{station_list_ny[x]}_elev.csv"
    )
    x += 1

In [ ]:
slope_df = stat_anal(sorted_direct, ny_df, station_list_ny, ny_df_lons, ny_df_lats)

In [ ]:
slope_df

In [ ]:
slope_df.to_csv("/home/aevans/nwp_bias/src/correlation/data/elev_gfs.csv")